In [ ]:
import plotly.graph_objects as go
import numpy as np
import random
import copy
import time
import math

## **Class implementation**

In [ ]:
# min in (0,0); huge set of local minima
def ackley(individual: list[float,float]) -> float: 
	x, y = individual
	fxy = -20 * math.exp( -0.2 * (0.5 * (x**2 + y**2)) ** 0.5 ) \
		- math.exp (0.5 * (math.cos(2 * math.pi * x) + math.cos(2 * math.pi * y))) + math.e + 20
	return fxy

def fitness_ackley(individual: list[float, float]):
	return 1 / (1 + ackley(individual))

In [ ]:
class AG_Ackley():
    def __init__(self, population_size: int):
        self.population = [[random.uniform(-32.768, 32.768) for _ in range(2)] for _ in range(population_size)]
        self.fitnesses = []

    def sort_pop(self, fitness_function, reverse_sort: bool) -> tuple[list[list], list]:
        """Sort population by fitness function. Return tuple with population list and fitness list"""

        fitness_list = [fitness_function(ind) for ind in self.population]
        lista = sorted(zip(self.population, fitness_list), key=lambda x: x[1], reverse=reverse_sort)
        self.population = [x[0] for x in lista]
        self.fitnesses = [x[1] for x in lista]

    def select(self, T: int) -> list[float, float]:
        """Return a copy of an indivudual by tournament selection. Population already ordered by fitness"""

        choices=random.choices(copy.copy(self.population), k=T)
        indices=[self.population.index(c) for c in choices]
        return self.population[np.argmin(indices)]

    def crossover(self, parent1: list[float, float], parent2: list[float, float], pcross: float) -> tuple[list,list]: 
        if random.random()<pcross:
            child1 = [parent1[0], parent2[1]]
            child2 = [parent1[1], parent2[0]]
        else:
            child1, child2 = parent1[:], parent2[:]
        return child1, child2

    def mutate(self, individual: list[float, float], pmut: float) -> list[float, float]:
        if random.random()<pmut:
            individual = [random.uniform(-32.768, 32.768), random.uniform(-32.768, 32.768)]
        return individual

    def evolve(self, fitness_function, pmut=0.1, pcross=0.7, ngen=100, T=2, trace=50, reverse_sort=False, elitism=False) -> None:
        """Evolution procedure. Initial population already created"""

        for i in range(ngen):
            new_pop = []
            self.sort_pop(fitness_function, reverse_sort)
            if elitism:
                new_pop.append(self.population[0])
                new_pop.append(self.population[1])
            while len(new_pop) != 100:   
                individual1 = self.select(T)
                individual2 = self.select(T)
                child1, child2 = self.crossover(individual1, individual2, pcross)
                mutated1 = self.mutate(child1, pmut)
                mutated2 = self.mutate(child2, pmut)
                new_pop.append(mutated1)
                new_pop.append(mutated2)
                
            self.population = [*new_pop] # make a copy

            if i % trace == 0 or i == ngen-1: # en la última gen se ordena
                self.sort_pop(fitness_function, reverse_sort)
                print(f"Nº gen: {i}, Best ind: {self.population[0]}, Best fitness: {self.fitnesses[0]:.3f}")

## **Visual test**

In [ ]:
def ackley_test(x, y):
    term1 = -20 * np.exp(-0.2 * np.sqrt(0.5 * (x**2 + y**2)))
    term2 = -np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y)))
    return term1 + term2 + np.e + 20

def plot_ackley(solution: list[float, float]):
    
    x = np.linspace(-32.768, 32.768, 100)
    y = np.linspace(-32.768, 32.768, 100)
    X, Y = np.meshgrid(x, y)
    Z = ackley_test(X, Y)

    # surface plot
    fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Jet', showscale=False, opacity=0.85)])

    # global minimum at (0, 0)
    global_min_x, global_min_y = 0, 0
    global_min_z = ackley_test(global_min_x, global_min_y)
    fig.add_trace(go.Scatter3d(
        x=[global_min_x], y=[global_min_y], z=[global_min_z],
        mode='markers',
        marker=dict(size=6, color='purple', symbol='circle'),
        name=f'Global Minimum ({global_min_x},{global_min_y})'
    ))

    # solution
    solution_x, solution_y = solution
    solution_z = ackley_test(solution_x, solution_y)
    fig.add_trace(go.Scatter3d(
        x=[solution_x], y=[solution_y], z=[solution_z],
        mode='markers',
        marker=dict(size=6, color='black', symbol='circle'),
        name=f'Solution ({solution_x:.3f},{solution_y:.3f})'
    ))

    # Update layout
    fig.update_layout(
        title='Ackley Function',
        scene=dict(
            xaxis_title='X-axis',
            yaxis_title='Y-axis',
            zaxis_title='Z-axis'
        ),
        margin=dict(l=0, r=0, b=0, t=40),  # Adjust margins for better fit
        width=700,
        height=550,
        showlegend=True,  # Enable legend
        legend=dict(
            x=1,  # Position legend at the top-right corner
            y=1,
            xanchor='right',
            yanchor='top'
        )
    )

    fig.show()
# ----------------------------------------
plot_ackley([1.5, -2.3])

## **---------------------------Tests---------------------------**

In [16]:
start = time.time()
genetic_algorithm = AG_Ackley(population_size=100) 
genetic_algorithm.evolve(fitness_function=fitness_ackley, pmut=0.1, ngen=2000, T=6, trace=200, reverse_sort=True)
minutos, segundos = divmod(time.time()-start, 60)
print(f"*******Tiempo transcurrido: {int(minutos)} minutos y {segundos:.2f} segundos*******")

plot_ackley(genetic_algorithm.population[0])

Nº gen: 0, Best ind: [-0.307, 1.12], Best fitness: 0.181
Nº gen: 200, Best ind: [-0.307, -0.307], Best fitness: 0.238
Nº gen: 400, Best ind: [-0.307, -0.307], Best fitness: 0.238
Nº gen: 600, Best ind: [0.0007314339575543727, 0.0007314339575543727], Best fitness: 0.997
Nº gen: 800, Best ind: [0.0007314339575543727, 0.0007314339575543727], Best fitness: 0.997
Nº gen: 1000, Best ind: [0.0007314339575543727, 0.0007314339575543727], Best fitness: 0.997
Nº gen: 1200, Best ind: [0.0007314339575543727, 0.0007314339575543727], Best fitness: 0.997
Nº gen: 1400, Best ind: [0.0007314339575543727, 0.0007314339575543727], Best fitness: 0.997
Nº gen: 1600, Best ind: [0.0007314339575543727, 0.0007314339575543727], Best fitness: 0.997
Nº gen: 1800, Best ind: [0.0007314339575543727, 0.0007314339575543727], Best fitness: 0.997
Nº gen: 1999, Best ind: [0.0007314339575543727, 0.0007314339575543727], Best fitness: 0.997
*******Tiempo transcurrido: 0 minutos y 0.97 segundos*******
